In [1]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

### Data  & SPLIT

In [2]:
train = pd.read_parquet('./data/train_preprocess_4.parquet')
# test = pd.read_parquet('./test.parquet')
test = pd.read_parquet('./data/test_preprocess_4.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

print(train.shape)
print(train.Delay.value_counts())
print()

from sklearn.model_selection  import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train.drop(columns=['ID', 'Delay', 'Delay_num']), train['Delay_num'], test_size=0.2, random_state=42)

print(train_x.shape, val_x.shape)
print(train_y.value_counts())
print(val_y.value_counts())

(377014, 20)
Not_Delayed    210001
Delayed         45000
Name: Delay, dtype: int64

(301611, 17) (75403, 17)
0    265614
1     35997
Name: Delay_num, dtype: int64
0    66400
1     9003
Name: Delay_num, dtype: int64


In [3]:
test_x = test.drop(columns=['ID'])
print(train.shape, test.shape)

(377014, 20) (1000000, 18)


### 모델적합 1 - GRID SEARCH
+ 최저의 Loss를 갖는 모델 적합

In [4]:
# custom loss 함수 정의 : fitting 시간 측정용도
from sklearn.metrics import make_scorer, log_loss
import sys
import time

def custom_scorer(true, pred, counter=[0], start_time=[None]):
    if start_time[0] is None:
        start_time[0] = time.time()

    counter[0] += 1

    if counter[0] % 10 == 0:
        elapsed_time = time.time() - start_time[0]
        progress = (counter[0] / total_fits) * 100
        sys.stdout.write(f"\r진행률: {progress:.2f}% | 걸린 시간: {elapsed_time:.2f}초")
        sys.stdout.flush()

    return -log_loss(true, pred)  # log_loss를 계산한 후 음수로 변환합니다.

custom_score = make_scorer(custom_scorer, greater_is_better=True, needs_proba=True)


#### 2-1 전처리 이용

In [10]:
# 2-1 전처리 이용
train = pd.read_parquet('./data/train_preprocess_2-1.parquet')
test = pd.read_parquet('./data/test_preprocess_2-1.parquet')


# grid search 1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier


# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "num_leaves": [7, 15, 31],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring="neg_log_loss",
    cv=5,
    verbose=1,
    n_jobs=-1,
)

# 그리드 서치를 수행
grid_search.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search.best_params_}")
print(f"최고의 점수: {-1 * grid_search.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm = grid_search.best_estimator_


Fitting 5 folds for each of 81 candidates, totalling 405 fits
최적의 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 31}
최고의 점수: 0.4395


In [14]:
# 2-1 전처리 이용
train = pd.read_parquet('./data/train_preprocess_2-1.parquet')
test = pd.read_parquet('./data/test_preprocess_2-1.parquet')


# grid search2 from grid search1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
param_grid2 = {
    "n_estimators": [150, 200, 250, 300],
    "max_depth": [5, 7, 9],
    "learning_rate": [0.05, 0.075, 0.1, 0.125],
    "num_leaves": [20, 30, 40],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search2 = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid2,
    scoring="neg_log_loss",
    cv=5,
    verbose=1,
    n_jobs=-1,
)

# 그리드 서치를 수행
grid_search2.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search2.best_params_}")
print(f"최고의 점수: {-1 * grid_search2.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm2 = grid_search2.best_estimator_


Fitting 5 folds for each of 144 candidates, totalling 720 fits
최적의 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'num_leaves': 30}
최고의 점수: 0.4391


In [14]:
# 2-1 전처리 이용
train = pd.read_parquet('./data/train_preprocess_2-1.parquet')
test = pd.read_parquet('./data/test_preprocess_2-1.parquet')



# grid search3 from grid search2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
param_grid3 = {
    "n_estimators": [ 300, 400, 500, 600],
    "max_depth": [ 7, 8, 9],
    "learning_rate": [0.075, 0.1, 0.125],
    "num_leaves": [25, 30, 35, 40, 45],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search3 = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid3,
    scoring="neg_log_loss",
    cv=5,
    verbose=1,
    n_jobs=-1,
)

# 그리드 서치를 수행
grid_search3.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search3.best_params_}")
print(f"최고의 점수: {-1 * grid_search3.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm3 = grid_search3.best_estimator_

Fitting 5 folds for each of 180 candidates, totalling 900 fits
최적의 파라미터: {'learning_rate': 0.075, 'max_depth': 7, 'n_estimators': 300, 'num_leaves': 45}
최고의 점수: 0.4379


#### 2-3 전처리 이용

In [32]:
# 2-3 전처리 이용
train = pd.read_parquet('./data/train_preprocess_2-3.parquet')
test = pd.read_parquet('./data/test_preprocess_2-3.parquet')

# grid search
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
param_grid3 = {
    "n_estimators": [ 300, 400, 500, 600],
    "max_depth": [ 7, 8, 9],
    "learning_rate": [0.075, 0.1, 0.125],
    "num_leaves": [25, 30, 35, 40, 45],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search3 = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid3,
    scoring="neg_log_loss",
    cv=5,
    verbose=1,
    n_jobs=-1,
)

# 그리드 서치를 수행
grid_search3.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search3.best_params_}")
print(f"최고의 점수: {-1 * grid_search3.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm3 = grid_search3.best_estimator_

Fitting 5 folds for each of 180 candidates, totalling 900 fits
최적의 파라미터: {'learning_rate': 0.075, 'max_depth': 8, 'n_estimators': 400, 'num_leaves': 35}
최고의 점수: 0.4381


#### 전처리3 이용

In [13]:
# 1회 제출용

# 전처리 3 이용
train = pd.read_parquet('./data/train_preprocess_3.parquet')
test = pd.read_parquet('./data/test_preprocess_3.parquet')

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42, learning_rate=0.075, max_depth=8, n_estimators=400, num_leaves=35)

# 모델 fit
lgbm.fit(train_x, train_y)

# 예측
y_pred = lgbm.predict_proba(test_x)

In [18]:
# 전처리 3 이용
train = pd.read_parquet('./data/train_preprocess_3.parquet')
test = pd.read_parquet('./data/test_preprocess_3.parquet')

# grid search
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
# param_grid = {
#     "n_estimators": [ 300, 400, 500, 600],
#     "max_depth": [ 7, 8, 9],
#     "learning_rate": [0.075, 0.1, 0.125],
#     "num_leaves": [25, 30, 35, 40, 45],
# }

param_grid = {
    "n_estimators": [ 300],
    "max_depth": [ 8],
    "learning_rate": [0.075],
    "num_leaves": [30],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
#     scoring="neg_log_loss",
    scoring=custom_score,
    cv=5,
    verbose=1,
    n_jobs=4,
)

# 그리드 서치를 수행
grid_search.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search.best_params_}")
print(f"최고의 점수: {-1 * grid_search.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
최적의 파라미터: {'learning_rate': 0.075, 'max_depth': 8, 'n_estimators': 300, 'num_leaves': 30}
최고의 점수: 0.1842


#### 전처리4

In [6]:
# 1회 제출용
from lightgbm import LGBMClassifier
# 전처리 3 이용
train = pd.read_parquet('./data/train_preprocess_4.parquet')
test = pd.read_parquet('./data/test_preprocess_4.parquet')

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42, learning_rate=0.075, max_depth=8, n_estimators=400, num_leaves=35)

# 모델 fit
lgbm.fit(train_x, train_y)

# 예측
y_pred = lgbm.predict_proba(test_x)

In [8]:
# 트리적합
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train_x, train_y)
y_pred = clf.predict_proba(test_x)

### 모델적합 2 - Bayesian opt

#### 2-3 전처리 이용

In [ ]:
# 전체 데이터로 적합
import lightgbm as lgb
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split


# LightGBM 데이터셋 객체 생성
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
train_data = lgb.Dataset(train_x, label=train_y)
test_x = test.drop(columns=['ID'])

# 목적 함수 정의
def lgbm_eval(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda):
    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_leaves": int(num_leaves),
        "learning_rate": learning_rate,
        "n_estimators": int(n_estimators),
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
    }
    cv_result = lgb.cv(params, train_data, nfold=5, seed=42, stratified=True, verbose_eval=None, metrics=["binary_logloss"])
    return -np.mean(cv_result["binary_logloss-mean"])

# 범위 설정
bounds = {
    "num_leaves": (20, 100),
    "learning_rate": (0.01, 0.3),
    "n_estimators": (10, 400),
    "subsample": (0.6, 1),
    "colsample_bytree": (0.6, 1),
    "reg_alpha": (0, 3),
    "reg_lambda": (0, 3),
}


In [16]:
# 1. 적합 1
# Bayesian optimization 객체 생성
optimizer = BayesianOptimization(lgbm_eval, bounds, random_state=42)

# 최적화
optimizer.maximize(init_points=10, n_iter=50) 

# 최적의 하이퍼파라미터
opt_params = optimizer.max["params"]
opt_params["num_leaves"] = int(opt_params["num_leaves"])
opt_params["n_estimators"] = int(opt_params["n_estimators"])

# 최적의 하이퍼파라미터로 모델 학습
model = lgb.train(opt_params, train_data)


|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 1         | -0.445    | 0.7498    | 0.2857    | 295.5     | 67.89     | 0.4681    | 0.468     | 0.6232    |


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 2         | -0.44     | 0.9465    | 0.1843    | 286.1     | 21.65     | 2.91      | 2.497     | 0.6849    |


...

C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 59        | -0.4466   | 0.7098    | 0.02182   | 269.9     | 31.33     | 2.861     | 2.717     | 0.9891    |


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 60        | -0.4402   | 0.7715    | 0.1657    | 275.2     | 37.26     | 0.904     | 0.06092   | 0.7215    |
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000712 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2020
[LightGBM] [Info] Number of data points in the train set: 255001, number of used features: 15
[LightGBM] [Info] Start training from score 0.176470


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [29]:
# 최적모델
print(model.params)
loss = optimizer.max['target'] * (-1)
print()
print(f"Optimized validation Log Loss: {loss:.4f}")

{'colsample_bytree': 0.6228528159010699, 'learning_rate': 0.14214163258010426, 'num_leaves': 41, 'reg_alpha': 1.3651742816388468, 'reg_lambda': 2.900464275084884, 'subsample': 0.6909715016372198, 'num_iterations': 304, 'early_stopping_round': None}

Optimized validation Log Loss: 0.4394


In [30]:
# 2. 적합 2
# Bayesian optimization 객체 생성
optimizer2 = BayesianOptimization(lgbm_eval, bounds, random_state=42)

# 최적화
optimizer2.maximize(init_points=20, n_iter=100)  # init_point : 하이퍼파라미터 공간에서 init_point개 랜덤으로 뽑아서 목적함수 평가 / # n_iter 그 이후에 반복횟수

# 최적의 하이퍼파라미터
opt_params = optimizer2.max["params"]
opt_params["num_leaves"] = int(opt_params["num_leaves"])
opt_params["n_estimators"] = int(opt_params["n_estimators"])

# 최적의 하이퍼파라미터로 모델 학습
model2 = lgb.train(opt_params, train_data)


|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 1         | -0.445    | 0.7498    | 0.2857    | 295.5     | 67.89     | 0.4681    | 0.468     | 0.6232    |


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 2         | -0.44     | 0.9465    | 0.1843    | 286.1     | 21.65     | 2.91      | 2.497     | 0.6849    |


...

C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 119       | -0.4424   | 0.6417    | 0.05619   | 302.5     | 23.47     | 2.981     | 0.3561    | 0.8443    |


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:573: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


| 120       | -0.441    | 0.925     | 0.09368   | 270.5     | 26.67     | 1.015     | 0.3894    | 0.6491    |
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2020
[LightGBM] [Info] Number of data points in the train set: 255001, number of used features: 15
[LightGBM] [Info] Start training from score 0.176470


C:\Users\Administrator\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [31]:
# 최적모델
print(model2.params)
loss = optimizer2.max['target'] * (-1)
print()
print(f"Optimized validation Log Loss: {loss:.4f}")

{'colsample_bytree': 0.7702765948396886, 'learning_rate': 0.10882839350091122, 'num_leaves': 57, 'reg_alpha': 2.827881419829328, 'reg_lambda': 0.8917347784084495, 'subsample': 0.8721630767409048, 'num_iterations': 362, 'early_stopping_round': None}

Optimized validation Log Loss: 0.4391


### 예측

In [11]:
#  1. grid search
y_pred = best_lgbm.predict_proba(test_x)

In [13]:
# 2. bayesian opt
test_x = test.drop(columns=['ID'])
y_pred = model.predict(test_x)
y_pred_inverse = 1 - y_pred
y_pred_final = np.column_stack((y_pred_inverse, y_pred ))

### 제출

In [9]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./submission/submission_semi-tree.csv', index=True)

In [20]:
submission2 = pd.DataFrame(data=y_pred_final, columns=sample_submission.columns, index=sample_submission.index)
submission2.to_csv('./submission/submission_lgbm_bayes6.csv', index=True)